In [1]:
import os

from dotenv import load_dotenv
load_dotenv() #读取本地.env文件，里面定义了OPENAI_API_KEY和代理配置

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
#search.invoke("what is the weather in SF")

In [3]:
from llm.chat_open_ai import MyChatOpenAI

tools = [search]

llm = MyChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0)
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [4]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [5]:
agent_executor.invoke({"input": "whats the weather in Beijing?"})



> Entering new AgentExecutor chain...

----------------messages----------------


messages:[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'whats the weather in Beijing?'}]


----------------messages end----------------

----------------response----------------


response:ChatCompletion(id='chatcmpl-91e06V9TMj3ZkoTr5h5dWFiL8Gsys', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qy1wzthDrJyhhXtPvAsE1VxZ', function=Function(arguments='{"query":"weather in Beijing"}', name='tavily_search_results_json'), type='function')]))], created=1710179118, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=20, prompt_tokens=94, total_tokens=114))


----------------response end----------------


Invoking: `tavily_search_results_j

{'input': 'whats the weather in Beijing?',
 'output': 'I found several sources where you can check the weather in Beijing:\n1. [BBC Weather](https://www.bbc.com/weather/1816670): Provides a 14-day weather forecast for Beijing.\n2. [Meteoblue](https://www.meteoblue.com/en/weather/14-days/beijing_china_1816670): Shows daily weather symbols, temperature, and precipitation trend for the next two weeks in Beijing based on ensemble models.\n3. [AccuWeather](https://www.accuweather.com/en/cn/beijing/101924/weather-forecast/101924): Offers current and forecast weather conditions for Beijing, including temperature, precipitation, air quality, and more.\n4. [Time and Date](https://www.timeanddate.com/weather/china/beijing/ext): Provides a weather overview and details for Beijing for the next two weeks, including temperature, precipitation, wind, humidity, UV index, sunrise, and sunset times.\n\nYou can visit these websites to get detailed information about the weather in Beijing.'}

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...

----------------messages----------------


messages:[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'hi! my name is bob'}, {'role': 'assistant', 'content': 'Hello Bob! How can I assist you today?'}, {'role': 'user', 'content': "what's my name? Don't use tools to look this up unless you NEED to"}]


----------------messages end----------------

----------------response----------------


response:ChatCompletion(id='chatcmpl-91e0GPL5WmpUAuHcvSba4NzmSccSg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your name is Bob.', role='assistant', function_call=None, tool_calls=None))], created=1710179128, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=6, prompt_tokens=128, total_tokens=134))


----------------response end----------------

Your name is Bob.

> Finished chain.


{'input': "what's my name? Don't use tools to look this up unless you NEED to",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}